In [1]:
import sys
import os

import numpy as nplk
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

sys.path.insert(0, '../../')
import ccal
%matplotlib inline
%config InlineBackend.figure_formats = {'svg',}

==================== Computational Cancer Analysis Library ====================

<11:06:25.635247> Checking dependencies ...
<11:06:25.637456> Using the following packages:
<11:06:25.639227> 	matplotlib (v1.5.1)
<11:06:25.639245> 	numpy (v1.10.4)
<11:06:25.639254> 	pandas (v0.18.0)
<11:06:25.639274> 	rpy2 (v2.7.9)
<11:06:25.639282> 	scikit-learn (v0.17.1)
<11:06:25.639289> 	scipy (v0.17.0)
<11:06:25.639296> 	seaborn (v0.7.0)


In [2]:
nfeatures = 2
nelement = 1
features = ccal.support.make_random_features(nfeatures, nelement)
ref = ccal.support.make_random_features(1, nelement)
scores = ccal.analyze.rank_features_against_reference(features, ref)

<11:06:26.063850> Computing features vs. Feature 0 using ic metric ...
<11:06:26.063933> Using 1 intersecting columns from features and ref, which have 1 and 1 columns respectively ...
Bootstrapping to get 0.95 confidence interval ...
[ nan  nan  nan]
[ nan  nan  nan]
Performing permutation test with 3 permutations ...


In [3]:
scores

,ic,0.95 CI,Local P-Value,Global P-Value,FDR (BH)
Feature 0,NaN,"<nan, nan>",0.333333,0.166667,0.166667
Feature 1,NaN,"<nan, nan>",0.333333,0.166667,0.166667


In [ ]:
import os
import math

import numpy as np
import pandas as pd
import scipy.stats as stats
from statsmodels.sandbox.stats.multicomp import multipletests
from scipy.spatial import distance
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import linkage, cophenet
from sklearn.decomposition import NMF
import matplotlib.pyplot as plt
import seaborn as sns

from ccal.visualize import CMAP_CONTINUOUS, plot_nmf_result, plot_features_and_reference, FONT16_BOLD, FONT12_BOLD, FONT20_BOLD, BLACK
from ccal.information import information_coefficient, cmi_diff, cmi_ratio
from ccal.support import verbose_print, establish_path